In [1]:
import sklearn
from sklearn.cluster import KMeans
import nltk
import numpy as np
import pickle
from nltk.tokenize import word_tokenize
from tqdm import tqdm

In [2]:
import tensorflow as tf
from training.model.config1 import Config
from training.model.ner_model1 import NERModel
from training.model.data_utils1 import CoNLLDataset, get_vocabs, UNK, NUM, \
    get_glove_vocab, write_vocab, load_vocab, get_char_vocab, \
    export_trimmed_glove_vectors, get_processing_word


In [3]:
#import files created in preprocessing
sent_embed_list = pickle.load(open ("sent_embed_list.p", "rb"))
sent_id_dict = pickle.load(open ("sent_id_dict.p", "rb"))
sent_list = pickle.load(open ("sent_list.p", "rb"))
sent_label_conf_pos_from_folds = pickle.load(open ("sent_label_conf_pos_from_folds.p", "rb"))
sent_with_num2orig_sent_dict = pickle.load(open ("sent_with_num2orig_sent_dict.p", "rb"))


embedding_len = 200

In [8]:
#initiate the KMeans input matrix
X = np.zeros((len(sent_id_dict),embedding_len), dtype=float)
#print(len(sent_id_dict))
print(X.shape)
print(X)

(40822, 200)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [9]:
#populate the KMeans input matrix with sentence embeddings created in preprocessing
for i in range(len(X)):
    X[i] = X[i] + sent_embed_list[i]

In [10]:
#check matrix dimentions 
print(X.shape)
#X = X[0:1000,:] #used for prototyping to create a smaller dataset
print(X.shape)

(40822, 200)
(40822, 200)


In [11]:
#initiate tf session
tf.reset_default_graph()
config = Config()
print('Config')    
model = NERModel(config)
model.build()

Config


/home/masha/anaconda3/envs/my_tensorflow/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
Initializing tf session


In [13]:
#tuning of two hyperparameters: 
#k - number of clusters for kMeans
#percentage - % of the cluster to be used as training data 

k_list = [10,100,200]
percentage_list = [10,30,50]


for k in k_list:
    for percentage in percentage_list:
        #create the training dataset based on kMeans clusters and prediction confidence
        sentences_for_training = []
        kmeans = KMeans(n_clusters=k, random_state=0).fit(X)
        labels = kmeans.predict(X)
        sent_cluster_dict = {}
        for i in range(len(X)):
            sent_cluster_dict[' '.join(sent_list[i])] = labels[i]
        cluster_sent_dict = {}
        for key, value in sorted(sent_cluster_dict.items()):
            cluster_sent_dict.setdefault(value, []).append(key)
        
        for key in cluster_sent_dict:
            conf_dict = {}
            for sent in cluster_sent_dict[key]:
                if len(sent) > 0:
                    conf_dict[sent] = sent_label_conf_pos_from_folds[sent][-1]['confidence']
            number_of_sent = round(float(len(cluster_sent_dict[key])) * float(percentage) / 100.0)
            if number_of_sent == 0:
                number_of_sent = 1 #have a minimum of 1 sent per cluster           
            training_sent_list = sorted(conf_dict.items(), key=lambda x:-x[1])[:number_of_sent]
            for sentence in training_sent_list:
                sentences_for_training.append(sentence[0])

        #make conll-like data based on the training data created above
        fout = open('kMeans_data/train_k=%s_percentage=%s.txt' %(k, percentage), 'w')
        for sentence in sentences_for_training:
            words = sentence.split(" ")
            pos = sent_label_conf_pos_from_folds[sentence][1]['pos_tags']
            labels = sent_label_conf_pos_from_folds[sentence][0]['labels']
            #print(words, len(words))
            #print(pos, len(pos))
            #print(labels, len(labels)) 
            #print('\n')
            for i in range(len(words)):
                fout.write('%s %s %s\n' %(words[i], pos[i], labels[i]))
            fout.write('\n')
        
        
        
        #build support data and train and evaluate using the created datset
        processing_word = get_processing_word(lowercase=True)
        print('Processing_word')
        train = CoNLLDataset('kMeans_data/train_k=%s_percentage=%s.txt' %(k, percentage), processing_word)
        dev   = CoNLLDataset('kMeans_data/dev.txt', processing_word)
        #test  = CoNLLDataset('kMeans_data/dev.txt', processing_word)

        vocab_words, vocab_tags = get_vocabs([train, dev])
        vocab_glove = get_glove_vocab(config.filename_glove)
        vocab = vocab_words & vocab_glove
        vocab.add(UNK)
        vocab.add(NUM)

        # Save vocab
        write_vocab(vocab, config.filename_words)
        write_vocab(vocab_tags, config.filename_tags)

        # Trim GloVe Vectors
        vocab = load_vocab(config.filename_words)
        export_trimmed_glove_vectors(vocab, config.filename_glove,
                                    config.filename_trimmed, config.dim_word)

        #Build and save char vocab
        train = CoNLLDataset('kMeans_data/train_k=%s_percentage=%s.txt' %(k, percentage))
        vocab_chars = get_char_vocab(train)
        write_vocab(vocab_chars, config.filename_chars)
        
        #training
       # config = Config(load=True)
        
        

        dev   = CoNLLDataset('kMeans_data/dev.txt', config.processing_word,
                         config.processing_tag, config.max_iter)
        train = CoNLLDataset('kMeans_data/train_k=%s_percentage=%s.txt' %(k, percentage), config.processing_word,
                         config.processing_tag, config.max_iter)
        # train model
        print('training')
        print("hyperparam for the training below: %", percentage, "k", k)
        model.train(train, dev)
        
        tf.reset_default_graph()
        config = Config()
        print('Config')    
        model = NERModel(config)
        model.build()
        
    
        

Processing_word
Building vocab...
- done. 25082 tokens
Building vocab...
- done. 400000 tokens
Writing vocab...
- done. 16056 tokens
Writing vocab...
- done. 4 tokens
Writing vocab...
- done. 84 tokens
training
hyperparam for the training below: % 10 k 10


Epoch 1 out of 15


204/204 [==============================] - 100s - train loss: 22.6613   


f1 0.49 - p 0.70 - r 0.44


7812.0 10454.0
o: f1=0.37  p=0.75  r=0.25
12690.0 18649.0
p: f1=0.53  p=0.68  r=0.43
167048.0 225803.0
N: f1=0.83  p=0.74  r=0.96
3434.0 5289.0
i: f1=0.23  p=0.65  r=0.14
Confusion matrix: 
 rows: true labels (the order is the same as in /data/tags.txt) 
 columns: predicted labels (the order is the same as in /data/tags.txt)
[[  7812    679  22853    291]
 [    90  12690  16156    303]
 [  2434   3949 167048   1261]
 [   118   1331  19746   3434]]


...

f1 0.66 - p 0.71 - r 0.63
Epoch 10 out of 15


12143.0 20059.0
i: f1=0.54  p=0.61  r=0.49
156156.0 191391.0
N: f1=0.85  p=0.82  r=0.89
14893.0 20755.0
o: f1=0.57  p=0.72  r=0.47
19454.0 27990.0
p: f1=0.68  p=0.70  r=0.67
Confusion matrix: 
 rows: true labels (the order is the same as in /data/tags.txt) 
 columns: predicted labels (the order is the same as in /data/tags.txt)
[[ 12143  10703    404   1379]
 [  6763 156156   5283   6490]
 [   429  15646  14893    667]
 [   724   8886    175  19454]]
1021/1021 [==============================] - 256s - train loss: 0.7825   


f1 0.67 - p 0.70 - r 0.65
- early stopping 3 epochs without improvement


13238.0 22884.0
i: f1=0.56  p=0.58  r=0.54
153191.0 185591.0
N: f1=0.85  p=0.83  r=0.88
15899.0 22785.0
o: f1=0.58  p=0.70  r=0.50
19710.0 28935.0
p: f1=0.68  p=0.68  r=0.67
Confusion matrix: 
 rows: true labels (the order is the same as in /data/tags.txt) 
 columns: predicted labels (the order is the same as in /data/tags.txt)
[[ 13238   9604    445   1342]
 [  8178 153191   6228   7095]
 [   521  14427  15899    788]
 [   947   8369    213  19710]]
Config


/home/masha/anaconda3/envs/my_tensorflow/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
Initializing tf session
